In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy.dialects.mssql.information_schema import columns
from sklearn.preprocessing import OneHotEncoder

# Experiment 7
In this experiment, apart from previous attempts, samples are to be split into train-test sets by the samples (Wood, Coal, Digestate, etc.), not random 80-20 split.

## Aim of the action
- As a result of regular 80-20 train-test-split approach, part of the data points from the same sample measurement at the same parameters fall into the test split (statistically, around one-third of the same sample at same conditions). This leads data leakage, and unreliable accuracy performance of model on test dataset.

## Details & Key Changes
### Outlier Removal
Biomass Data Before: 740 	After: 733	 Removed: 7
Mix Data Before:	 563 	After: 555	 Removed: 8
Coal Data Before:	 195 	After: 191	 Removed: 4
Plastic Data Before: 492 	After: 482	 Removed: 10

- test data contains **biomass, mix, plastic** fuels 'wood, digestate_pe, gummipulver)
- max r2: 0.7387 with LightGBM (Baselines w/o hyperparameter tuning)


## Comments | To be improved
- Other fuel combinations are to be selected as test set.

In [2]:
features = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\preprocess\features.csv", delimiter=';')
labels = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\preprocess\labels.csv", delimiter=';')

In [3]:
features.columns

Index(['sample', 'wc_1', 'vm_1', 'fc_1', 'ac_1', 'c_1', 'h_1', 'o_1', 'n_1',
       's_1', 'cl_1', 'hc_1', 'oc_1', 'lhv_1', 'wc_2', 'vm_2', 'fc_2', 'ac_2',
       'c_2', 'h_2', 'o_2', 'n_2', 's_2', 'cl_2', 'hc_2', 'oc_2', 'lhv_2',
       'atmosphere', 'temperature', 'residence_time', 'pressure', 'heat_rate',
       'x_fuel1', 'x_fuel2'],
      dtype='object')

In [4]:
wc = features['wc_1']*features['x_fuel1'] + features['wc_2']*features['x_fuel2']
vm = features['vm_1']*features['x_fuel1'] + features['vm_2']*features['x_fuel2']
fc = features['fc_1']*features['x_fuel1'] + features['fc_2']*features['x_fuel2']
ac = features['ac_1']*features['x_fuel1'] + features['ac_2']*features['x_fuel2']
c = features['c_1']*features['x_fuel1'] + features['c_2']*features['x_fuel2']
h = features['h_1']*features['x_fuel1'] + features['h_2']*features['x_fuel2']
o = features['o_1']*features['x_fuel1'] + features['o_2']*features['x_fuel2']
n = features['n_1']*features['x_fuel1'] + features['n_2']*features['x_fuel2']
s = features['s_1']*features['x_fuel1'] + features['s_2']*features['x_fuel2']
cl = features['cl_1']*features['x_fuel1'] + features['cl_2']*features['x_fuel2']
hc = features['hc_1']*features['x_fuel1'] + features['hc_2']*features['x_fuel2']
oc = features['oc_1']*features['x_fuel1'] + features['oc_2']*features['x_fuel2']
lhv = features['lhv_1']*features['x_fuel1'] + features['lhv_2']*features['x_fuel2']

In [5]:
features.drop(['wc_1', 'vm_1', 'fc_1', 'ac_1', 'c_1', 'h_1', 'o_1', 'n_1',
       's_1', 'cl_1', 'hc_1', 'oc_1', 'lhv_1', 'wc_2', 'vm_2', 'fc_2', 'ac_2',
       'c_2', 'h_2', 'o_2', 'n_2', 's_2', 'cl_2', 'hc_2', 'oc_2', 'lhv_2',
       'atmosphere','x_fuel1', 'x_fuel2'], axis=1, inplace=True)

In [6]:
features = pd.concat([features, wc, vm, fc, ac, c,h,o,n,s,cl,hc,oc,lhv ],  axis=1)


In [7]:
features.columns = ['sample', 'temperature', 'residence_time', 'pressure',
            'heat_rate', 'wc', 'vm', 'fc', 'ac', 'c', 'h', 'o', 'n', 's', 'cl', 'hc', 'oc', 'lhv']

In [8]:
features.head()

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,o,n,s,cl,hc,oc,lhv
0,brown_coal_1,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
1,brown_coal_2,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
2,brown_coal_3,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
3,brown_coal_4,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
4,brown_coal_5,600,1.0,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13


In [9]:
def classify_fuel(sample):
    sample = str(sample).lower()

    if "gumm" in sample or "gm" in sample:
        return "gummipulver"
    elif "wod" in sample or 'wood' in sample or 'wd' in sample:
        return "wood"
    elif "coal" in sample or 'bks' in sample:
        return "coal"
    elif "swage" in sample or 'swg' in sample or 'sewage' in sample:
        return "sewage"
    elif "refuse" in sample or 'ref' in sample:
        return "ebs2"
    elif "digestate" in sample or 'dgst' in sample or "dgt" in sample:
        if 'pe' not in sample and 'pp' not in sample and 'car' not in sample:
            return "digestate"
        if 'pe' in sample:
            return "digestate_pe"
        if 'car' in sample:
            return "digestate_car"
        if 'pp' in sample:
            return "digestate_pp"
    elif "htc" in sample:
        return "HTC"
    elif "cel" in sample:
        return "cellulose"
    elif "hc" in sample:
        return "Hemicellulose"
    elif "lig" in sample:
        return "Lignin"
    elif "ebs2" in sample:
        return "ebs2"
    elif "ebs1" in sample:
        return "ebs1"
    else:
        return "unknown"  # Default category


features['fuel_type'] = features['sample'].apply(classify_fuel)

print(features.head())

         sample  temperature  residence_time  pressure  heat_rate    wc    vm  \
0  brown_coal_1          600             0.5       1.0       1000  10.8  44.2   
1  brown_coal_2          600             0.5       1.0       1000  10.8  44.2   
2  brown_coal_3          600             0.5       1.0       1000  10.8  44.2   
3  brown_coal_4          600             0.5       1.0       1000  10.8  44.2   
4  brown_coal_5          600             1.0       1.0       1000  10.8  44.2   

     fc   ac     c    h     o    n    s   cl   hc   oc    lhv fuel_type  
0  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
1  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
2  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
3  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
4  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  


In [10]:
def fuel_category(sample):
    sample = str(sample).lower()

    if "gumm" in sample or "gm" in sample:
        return "plastic"
    elif "wod" in sample or 'wood' in sample or 'wd' in sample:
        return "biomass"
    elif "coal" in sample or 'bks' in sample:
        return "coal"
    elif "swage" in sample or 'swg' in sample or 'sewage' in sample:
        return "biomass"
    elif "refuse" in sample or 'ref' in sample:
        return "plastic"
    elif "digestate" in sample or 'dgst' in sample or "dgt" in sample:
        if 'pe' not in sample and 'pp' not in sample and 'car' not in sample:
            return "plastic"
        if 'pe' in sample:
            return "mix"
        if 'car' in sample:
            return "mix"
        if 'pp' in sample:
            return "mix"
    elif "htc" in sample:
        return "biomass"
    elif "cel" in sample:
        return "biomass"
    elif "hc" in sample:
        return "biomass"
    elif "lig" in sample:
        return "biomass"
    elif "ebs2" in sample:
        return "plastic"
    elif "ebs1" in sample:
        return "plastic"
    else:
        return "unknown"  # Default category

features['fuel_category'] = features['sample'].apply(fuel_category)

In [11]:
set(features['fuel_category'])

{'biomass', 'coal', 'mix', 'plastic'}

In [12]:
features.head

<bound method NDFrame.head of             sample  temperature  residence_time  pressure  heat_rate    wc  \
0     brown_coal_1          600             0.5       1.0       1000  10.8   
1     brown_coal_2          600             0.5       1.0       1000  10.8   
2     brown_coal_3          600             0.5       1.0       1000  10.8   
3     brown_coal_4          600             0.5       1.0       1000  10.8   
4     brown_coal_5          600             1.0       1.0       1000  10.8   
...            ...          ...             ...       ...        ...   ...   
1985         gm_39         1000             1.0      10.0         10   2.6   
1986         gm_40         1000             1.0      10.0         50   2.6   
1987         gm_41         1000             1.0      10.0        100   2.6   
1988         gm_42         1000             1.0      10.0        500   2.6   
1989         gm_43         1000             1.0      10.0       1000   2.6   

        vm    fc    ac     c    h

In [13]:
features.isnull().sum()

sample              0
temperature         0
residence_time      0
pressure            0
heat_rate           0
wc                  0
vm                  0
fc                  0
ac                  0
c                   0
h                   0
o                   0
n                   0
s                   0
cl                  0
hc                  0
oc                  0
lhv               206
fuel_type           0
fuel_category       0
dtype: int64

In [14]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(features[['fuel_category']])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out())
features_encoded = pd.concat([features, one_hot_df], axis=1)
features_encoded.drop(columns=['fuel_category'], inplace=True)
print(features_encoded.head())


         sample  temperature  residence_time  pressure  heat_rate    wc    vm  \
0  brown_coal_1          600             0.5       1.0       1000  10.8  44.2   
1  brown_coal_2          600             0.5       1.0       1000  10.8  44.2   
2  brown_coal_3          600             0.5       1.0       1000  10.8  44.2   
3  brown_coal_4          600             0.5       1.0       1000  10.8  44.2   
4  brown_coal_5          600             1.0       1.0       1000  10.8  44.2   

     fc   ac     c  ...    s   cl   hc   oc    lhv  fuel_type  \
0  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
1  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
2  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
3  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
4  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   

   fuel_category_biomass  fuel_category_coal fuel_category_mix  \
0                    0.0                 1.0            

In [15]:
features_encoded = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\!Experiment_1\features_encoded.csv")

In [16]:
fuel_type_counts = features_encoded['fuel_type'].value_counts()
print(fuel_type_counts)

fuel_type
sewage           241
digestate        236
wood             236
digestate_car    206
coal             195
digestate_pp     183
digestate_pe     174
ebs2             138
HTC              134
ebs1              62
gummipulver       56
Hemicellulose     44
Lignin            44
cellulose         41
Name: count, dtype: int64


In [17]:
data = pd.concat([features_encoded, labels['devol_yield']], axis=1)

In [18]:
data.columns

Index(['sample', 'temperature', 'residence_time', 'pressure', 'heat_rate',
       'wc', 'vm', 'fc', 'ac', 'c', 'h', 'o', 'n', 's', 'cl', 'hc', 'oc',
       'lhv', 'fuel_type', 'fuel_category_biomass', 'fuel_category_coal',
       'fuel_category_mix', 'fuel_category_plastic', 'devol_yield'],
      dtype='object')

In [19]:
def remove_outliers_iqr(group, cols, multiplier=1.5):
    # If cols is a string, convert it to a list
    if isinstance(cols, str):
        cols = [cols]
    Q1 = group[cols].quantile(0.25)
    Q3 = group[cols].quantile(0.75)
    IQR = Q3 - Q1
    # Keep rows where all specified columns are within the acceptable range
    mask = ~((group[cols] < (Q1 - multiplier * IQR)) | (group[cols] > (Q3 + multiplier * IQR))).any(axis=1)
    return group[mask]

# Assuming `data` is your DataFrame:
biomass_data = data[data['fuel_category_biomass'] == 1]
mix_data = data[data['fuel_category_mix'] == 1]
coal_data = data[data['fuel_category_coal'] == 1]
plastic_data = data[data['fuel_category_plastic'] == 1]

# Group each category by 'temperature'
biomass = biomass_data.groupby('temperature')
mix = mix_data.groupby('temperature')
coal = coal_data.groupby('temperature')
plastic = plastic_data.groupby('temperature')

biomass_clean = biomass.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
mix_clean = mix.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
coal_clean = coal.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
plastic_clean = plastic.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)

data_clean = data.groupby('temperature').apply(lambda g: remove_outliers_iqr(g, 'devol_yield'))

C:\Users\demir\AppData\Local\Temp\ipykernel_7592\3437265364.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  biomass_clean = biomass.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
C:\Users\demir\AppData\Local\Temp\ipykernel_7592\3437265364.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mix_clean = mix.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index

In [20]:
print(f'Biomass Data Before: {biomass_data.shape[0]}', f'\tAfter: {biomass_clean.shape[0]}\t Removed: {biomass_data.shape[0] - biomass_clean.shape[0]}')
print(f'Mix Data Before:\t {mix_data.shape[0]}', f'\tAfter: {mix_clean.shape[0]}\t Removed: {mix_data.shape[0] - mix_clean.shape[0]}')
print(f'Coal Data Before:\t {coal_data.shape[0]}', f'\tAfter: {coal_clean.shape[0]}\t Removed: {coal_data.shape[0] - coal_clean.shape[0]}')
print(f'Plastic Data Before: {plastic_data.shape[0]}', f'\tAfter: {plastic_clean.shape[0]}\t Removed: {plastic_data.shape[0] - plastic_clean.shape[0]}')

Biomass Data Before: 740 	After: 733	 Removed: 7
Mix Data Before:	 563 	After: 555	 Removed: 8
Coal Data Before:	 195 	After: 191	 Removed: 4
Plastic Data Before: 492 	After: 482	 Removed: 10


In [21]:
clean_data = pd.concat([biomass_clean, mix_clean, coal_clean, plastic_clean])
clean_data.head()

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,cl,hc,oc,lhv,fuel_type,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,htc_96,200,0.5,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,6.785137
1,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,2.878561
2,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,2.878561
3,cel_18,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,1.345032,1.020847,16.51605,17.562938,cellulose,1.0,0.0,0.0,0.0,2.018886
4,cel_19,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,1.345032,1.020847,16.51605,17.562938,cellulose,1.0,0.0,0.0,0.0,2.324709


In [22]:
fuel_type_counts_after = clean_data['fuel_type'].value_counts()
print(f'Before:{fuel_type_counts}\tAfter: {fuel_type_counts_after}')

Before:fuel_type
sewage           241
digestate        236
wood             236
digestate_car    206
coal             195
digestate_pp     183
digestate_pe     174
ebs2             138
HTC              134
ebs1              62
gummipulver       56
Hemicellulose     44
Lignin            44
cellulose         41
Name: count, dtype: int64	After: fuel_type
sewage           241
wood             236
digestate        233
digestate_car    205
coal             191
digestate_pp     181
digestate_pe     169
ebs2             133
HTC              131
ebs1              60
gummipulver       56
Lignin            44
Hemicellulose     41
cellulose         40
Name: count, dtype: int64


# Train-Test Split
- **coal** -> Test Data
- 90-10 train-test split


In [23]:
train_data = clean_data[(clean_data['fuel_type'] != 'wood')&(clean_data['fuel_type'] != 'digestate_pe')&(clean_data['fuel_type'] != 'gummipulver')].drop(columns=['fuel_type']).reset_index(drop=True)
print(clean_data.shape)
train_data.describe()


(1961, 24)


,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1295.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,798.726667,5.264333,2.327000,915.006667,6.639538,59.379159,14.748847,19.245626,47.586362,5.646712,...,0.655510,0.347099,1.380741,1.918474,19.913237,0.331333,0.127333,0.257333,0.284000,49.984140
std,191.988763,4.399189,3.902229,261.950272,2.909698,13.515274,11.490490,12.504887,13.135057,1.898219,...,0.666539,0.618324,0.362077,5.552783,6.686390,0.470850,0.333457,0.437310,0.451087,20.371017
min,200.000000,0.500000,0.500000,10.000000,2.700000,44.200000,3.064326,0.000000,32.400000,3.265000,...,0.046420,0.000000,0.461416,0.100000,13.528000,0.000000,0.000000,0.000000,0.000000,-2.871983
25%,600.000000,1.000000,1.000000,1000.000000,3.200000,50.700000,6.900000,8.200000,37.500000,4.517670,...,0.200000,0.000000,1.200000,0.200000,13.943000,0.000000,0.000000,0.000000,0.000000,37.291902
50%,800.000000,5.000000,1.000000,1000.000000,5.810000,53.700000,11.550000,19.580004,44.084545,4.800000,...,0.369425,0.000000,1.500000,0.326959,17.562938,0.000000,0.000000,0.000000,0.000000,52.219389
75%,1000.000000,10.000000,1.000000,1000.000000,10.200000,63.025000,15.400000,23.700000,61.534000,6.961905,...,0.800000,0.003570,1.625000,0.400000,22.868500,1.000000,0.000000,1.000000,1.000000,64.168190
max,1200.000000,20.000000,20.000000,1000.000000,10.800000,91.735674,40.900000,37.520000,68.700000,9.784200,...,2.100000,1.585535,1.760000,26.102250,32.177000,1.000000,1.000000,1.000000,1.000000,99.756944


In [24]:
train_data

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,htc_96,200,0.5,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,6.785137
1,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,2.878561
2,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,2.878561
3,cel_18,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,0.04642,1.345032,1.020847,16.51605,17.562938,1.0,0.0,0.0,0.0,2.018886
4,cel_19,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,0.04642,1.345032,1.020847,16.51605,17.562938,1.0,0.0,0.0,0.0,2.324709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,digestate_308,1200,10.0,1.0,1000,10.200,50.7000,15.4000,23.70,37.500000,...,0.20000,0.000000,1.500000,0.40000,15.055000,0.0,0.0,0.0,1.0,59.139403
1496,digestate_309,1200,10.0,1.0,1000,10.200,50.7000,15.4000,23.70,37.500000,...,0.20000,0.000000,1.500000,0.40000,15.055000,0.0,0.0,0.0,1.0,60.374640
1497,digestate_310,1200,10.0,1.0,1000,10.200,50.7000,15.4000,23.70,37.500000,...,0.20000,0.000000,1.500000,0.40000,15.055000,0.0,0.0,0.0,1.0,54.277600
1498,digestate_311,1200,10.0,1.0,1000,10.200,50.7000,15.4000,23.70,37.500000,...,0.20000,0.000000,1.500000,0.40000,15.055000,0.0,0.0,0.0,1.0,59.020408


In [25]:
test_data = clean_data[
    clean_data['fuel_type'].isin(['wood', 'digestate_pe','gummipulver'])
].drop(columns=['fuel_type']).reset_index(drop=True)

In [26]:
test_data

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,wood_8,400,10.0,1.0,1000,7.6,75.4,16.4,0.6,50.4,...,0.1,0.0,1.4,0.5,19.843,1.0,0.0,0.0,0.0,7.343499
1,wood_10,400,10.0,1.0,1000,7.6,75.4,16.4,0.6,50.4,...,0.1,0.0,1.4,0.5,19.843,1.0,0.0,0.0,0.0,8.267581
2,wood_11,400,10.0,1.0,1000,7.6,75.4,16.4,0.6,50.4,...,0.1,0.0,1.4,0.5,19.843,1.0,0.0,0.0,0.0,6.379822
3,wood_12,400,10.0,1.0,1000,7.6,75.4,16.4,0.6,50.4,...,0.1,0.0,1.4,0.5,19.843,1.0,0.0,0.0,0.0,4.096141
4,wood_1,400,10.0,1.0,1000,7.6,75.4,16.4,0.6,50.4,...,0.1,0.0,1.4,0.5,19.843,1.0,0.0,0.0,0.0,27.319384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,gm_40,1000,1.0,10.0,50,2.6,58.9,27.6,10.9,82.3,...,2.5,0.1,1.0,0.0,35.800,0.0,0.0,0.0,1.0,68.838085
457,gm_41,1000,1.0,10.0,100,2.6,58.9,27.6,10.9,82.3,...,2.5,0.1,1.0,0.0,35.800,0.0,0.0,0.0,1.0,68.634051
458,gm_42,1000,1.0,10.0,500,2.6,58.9,27.6,10.9,82.3,...,2.5,0.1,1.0,0.0,35.800,0.0,0.0,0.0,1.0,69.513635
459,gm_43,1000,1.0,10.0,1000,2.6,58.9,27.6,10.9,82.3,...,2.5,0.1,1.0,0.0,35.800,0.0,0.0,0.0,1.0,69.260096


In [27]:
X_train = train_data.drop(columns=['sample', 'devol_yield'])
y_train = train_data['devol_yield']

X_test = test_data.drop(columns=['sample', 'devol_yield'])
y_test = test_data['devol_yield']

# kNN Imputation & Standard Scaling

In [28]:
import shap
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
X_train.isnull().sum()

temperature                0
residence_time             0
pressure                   0
heat_rate                  0
wc                         0
vm                         0
fc                         0
ac                         0
c                          0
h                          0
o                          0
n                          0
s                          0
cl                         0
hc                         0
oc                         0
lhv                      205
fuel_category_biomass      0
fuel_category_coal         0
fuel_category_mix          0
fuel_category_plastic      0
dtype: int64

In [30]:
knn_imputer = KNNImputer(n_neighbors=3)
X_train_imputed = knn_imputer.fit_transform(X_train)
X_test_imputed = knn_imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_scaled

,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,...,n,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic
0,-3.119591,-1.083364,-0.340175,0.324572,0.242533,-0.824465,-0.683301,1.461866,-1.104388,-1.255127,...,-0.033075,-0.470513,-0.561541,-0.499346,-0.273553,-0.987775,1.420602,-0.381985,-0.588642,-0.629800
1,-3.119591,-0.969669,-0.340175,0.324572,0.242533,-0.824465,-0.683301,1.461866,-1.104388,-1.255127,...,-0.033075,-0.470513,-0.561541,-0.499346,-0.273553,-0.987775,1.420602,-0.381985,-0.588642,-0.629800
2,-3.119591,-0.969669,-0.340175,0.324572,0.242533,-0.824465,-0.683301,1.461866,-1.104388,-1.255127,...,-0.033075,-0.470513,-0.561541,-0.499346,-0.273553,-0.987775,1.420602,-0.381985,-0.588642,-0.629800
3,-3.119591,-0.969669,-0.340175,0.324572,-0.314412,2.111008,-0.729050,-1.539562,-0.554500,-0.594988,...,-1.562027,-0.914116,1.614469,-0.994304,2.629753,-0.359495,1.420602,-0.381985,-0.588642,-0.629800
4,-3.119591,-0.969669,-0.340175,0.324572,-0.314412,2.111008,-0.729050,-1.539562,-0.554500,-0.594988,...,-1.562027,-0.914116,1.614469,-0.994304,2.629753,-0.359495,1.420602,-0.381985,-0.588642,-0.629800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2.090785,1.076845,-0.340175,0.324572,1.224062,-0.642388,0.056688,0.356329,-0.768152,-0.446205,...,0.464420,-0.683624,-0.561541,0.329483,-0.273553,-0.750006,-0.703927,-0.381985,-0.588642,1.587806
1496,2.090785,1.076845,-0.340175,0.324572,1.224062,-0.642388,0.056688,0.356329,-0.768152,-0.446205,...,0.464420,-0.683624,-0.561541,0.329483,-0.273553,-0.750006,-0.703927,-0.381985,-0.588642,1.587806
1497,2.090785,1.076845,-0.340175,0.324572,1.224062,-0.642388,0.056688,0.356329,-0.768152,-0.446205,...,0.464420,-0.683624,-0.561541,0.329483,-0.273553,-0.750006,-0.703927,-0.381985,-0.588642,1.587806
1498,2.090785,1.076845,-0.340175,0.324572,1.224062,-0.642388,0.056688,0.356329,-0.768152,-0.446205,...,0.464420,-0.683624,-0.561541,0.329483,-0.273553,-0.750006,-0.703927,-0.381985,-0.588642,1.587806


# Baseline Models
## Simple Baselines

In [31]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

# Dummy models
mean_model = DummyRegressor(strategy="mean")
median_model = DummyRegressor(strategy="median")

# KNN Regression
knn_model = KNeighborsRegressor(n_neighbors=5)

mean_model.fit(X_train_scaled, y_train)
median_model.fit(X_train_scaled, y_train)
knn_model.fit(X_train_scaled, y_train)
mean_predictions = mean_model.predict(X_test_scaled)
median_predictions = median_model.predict(X_test_scaled)
knn_predictions = knn_model.predict(X_test_scaled)

print('Mean Model Score:', mean_model.score(X_test_scaled, y_test))
print('Median Model Score:', median_model.score(X_test_scaled, y_test))
print('KNN Model Score:', knn_model.score(X_test_scaled, y_test))
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Mean Model MAE:", mean_absolute_error(y_test, mean_predictions))
print("Median Model MAE:", mean_absolute_error(y_test, median_predictions))
print("KNN Model MAE:", mean_absolute_error(y_test, knn_predictions))

Mean Model Score: -0.17604796134647427
Median Model Score: -0.10934268289476212
KNN Model Score: 0.5598060407316934
Mean Model MAE: 24.2403758008399
Median Model MAE: 23.385225754017355
KNN Model MAE: 13.485470709047288


## 2. Linear Models
Linear Regression (OLS)

Ridge Regression (L2 regularization)

Lasso Regression (L1 regularization)

Elastic Net (Combination of L1 and L2)

In [32]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

lin_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=0.1)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5)
lin_model.fit(X_train_scaled, y_train)
ridge_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)
elastic_model.fit(X_train_scaled, y_train)

print('Linear Model Score:', lin_model.score(X_test_scaled, y_test))
print('Ridge Model Score:', ridge_model.score(X_test_scaled, y_test))
print('Lasso Model Score:', lasso_model.score(X_test_scaled, y_test))
print('Elastic Model Score:', elastic_model.score(X_test_scaled, y_test))

Linear Model Score: 0.5012901896833193
Ridge Model Score: 0.5486098787188384
Lasso Model Score: 0.5260110070418911
Elastic Model Score: 0.5074019110980915


## 3. Tree-Based Models
Decision Tree Regression: Simple non-linear model.

Random Forest Regression: Ensemble of decision trees.

Gradient Boosting Regression (GBR)

XGBoost

LightGBM
CatBoost

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

dt_model = DecisionTreeRegressor(max_depth=5)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5)
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
dt_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)
gbr_model.fit(X_train_scaled, y_train)
xgb_model.fit(X_train_scaled, y_train)
lgb_model.fit(X_train_scaled, y_train)
dt_predictions = dt_model.predict(X_test_scaled)
gbr_predictions = gbr_model.predict(X_test_scaled)
lgb_predictions = lgb_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)
rf_predictions = rf_model.predict(X_test_scaled)
print('Decision Tree Regressor Score:', mean_squared_error(y_test, dt_predictions))
print('Gradient Boosting Regressor Score:', mean_squared_error(y_test, gbr_predictions))
print('lightGBM Regressor Score:', mean_squared_error(y_test, lgb_predictions))
print('xgboost Regressor Score:', mean_squared_error(y_test, xgb_predictions))
print('RF Regressor Score:', mean_squared_error(y_test, rf_predictions))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1240
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 21
[LightGBM] [Info] Start training from score 49.984140
Decision Tree Regressor Score: 308.7318137618574
Gradient Boosting Regressor Score: 173.57000282596755
lightGBM Regressor Score: 165.09272432656792
xgboost Regressor Score: 171.37666960381944
RF Regressor Score: 216.27707324743807


In [34]:
print('Decision Tree Regressor Score:', dt_model.score(X_test_scaled, y_test))
print('Gradient Boosting Regressor Score:', gbr_model.score(X_test_scaled, y_test))
print('lightGBM Regressor Score:', lgb_model.score(X_test_scaled, y_test))
print('XGBoost Regressor Score:', xgb_model.score(X_test_scaled, y_test))
print('RF Regressor Score:', rf_model.score(X_test_scaled, y_test))

Decision Tree Regressor Score: 0.5115309931114942
Gradient Boosting Regressor Score: 0.7253811783341708
lightGBM Regressor Score: 0.7387937507518392
XGBoost Regressor Score: 0.7288514242014291
RF Regressor Score: 0.6578109463528805


## 4. Nonlinear & Probabilistic Models
Gaussian Process Regression (GPR)

Support Vector Regression (SVR)

Neural Networks (MLP)

In [35]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

gpr_model = GaussianProcessRegressor()
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', max_iter=2000)
gpr_model.fit(X_train_scaled, y_train)
svr_model.fit(X_train_scaled, y_train)
mlp_model.fit(X_train_scaled, y_train)
print('Gaussian Process Regressor Score:', gpr_model.score(X_test_scaled, y_test))
print('SVR Score:', svr_model.score(X_test_scaled, y_test))
print('MLP Score:', mlp_model.score(X_test_scaled, y_test))

Gaussian Process Regressor Score: -809.50473025445
SVR Score: 0.34423960753083693
MLP Score: 0.1208631561910587
